In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
import sklearn.metrics as mt
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [ ]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [ ]:
#Reading data:

df = pd.read_csv("../input/kuhar/3.Time_domain_subsamples/KU-HAR_time_domain_subsamples_20750x300.csv",header=None)
dff = df.values
signals = dff[:, 0: 1800]                         #These are the time-domian subsamples (signals) 
signals = np.array(signals, dtype=np.float32)
labels = dff[:, 1800]                             #These are their associated class labels (signals)

print(signals.shape)
print(labels.shape)

In [ ]:
# Visualization of the 20001th (time-domain HAR) sample (channel data):
# The correspondig activity is "Jump"

Accelerometer_X_axis_data = signals[20000, 0: 300]
Accelerometer_Y_axis_data = signals[20000, 300: 600]
Accelerometer_Z_axis_data = signals[20000, 600: 900]
Gyroscope_X_axis_data = signals[20000, 900: 1200]
Gyroscope_Y_axis_data = signals[20000, 1200: 1500]
Gyroscope_Z_axis_data = signals[20000, 1500: 1800]
time = np.linspace(.01, 3, 300)

figure(figsize=(8, 30), dpi=80)

ax1 = plt.subplot(611)
ax1.plot(time, Accelerometer_X_axis_data, 'b')
ax1.title.set_text('Accelerometer X axis')
ax1.set_xlabel('time (ms) ->')
ax1.set_ylabel('Acceleration (m/s^2)')
ax1.grid(True)

ax2 = plt.subplot(612)
ax2.plot(time, Accelerometer_Y_axis_data, 'g')
ax2.title.set_text('Accelerometer Y axis')
ax2.set_xlabel('time (ms) ->')
ax2.set_ylabel('Acceleration (m/s^2)')
ax2.grid(True)

ax3 = plt.subplot(613)
ax3.plot(time, Accelerometer_Z_axis_data, 'r')
ax3.title.set_text('Accelerometer Z axis')
ax3.set_xlabel('time (ms) ->')
ax3.set_ylabel('Acceleration (m/s^2)')
ax3.grid(True)

ax4 = plt.subplot(614)
ax4.plot(time, Gyroscope_X_axis_data, 'b')
ax4.title.set_text('Gyroscope X axis')
ax4.set_xlabel('time (ms) ->')
ax4.set_ylabel('Angular rotation (rad/s)')
ax4.grid(True)

ax5 = plt.subplot(615)
ax5.plot(time, Gyroscope_Y_axis_data, 'g')
ax5.title.set_text('Gyroscope Y axis')
ax5.set_xlabel('time (ms) ->')
ax5.set_ylabel('Angular rotation (rad/s)')
ax5.grid(True)

ax6 = plt.subplot(616)
ax6.plot(time, Gyroscope_Z_axis_data, 'r')
ax6.title.set_text('Gyroscope Z axis')
ax6.set_xlabel('time (ms) ->')
ax6.set_ylabel('Angular rotation (rad/s)')
ax6.grid(True)

plt.show()

In [ ]:
# DFT of the time-domain signals, separately for each channel

fft = np.zeros(signals.shape, dtype=np.float32)
for i in range(0,len(signals)):
    for j in range(0, 6):
        tmp = np.fft.fft(signals[i, j*300:(j+1)*300])
        fft[i, j*300:(j+1)*300] = abs(tmp)
        
print(fft.shape)

In [ ]:
# Creating Training and Test subsets with randomly chosen samples:

X_train, X_test, y_train, y_test=train_test_split(fft,labels, test_size=0.3, random_state=0, stratify=labels)
print(X_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
X_train= st_x.fit_transform(X_train)    
X_test= st_x.transform(X_test)  

In [ ]:
accuracy_list = []
f1_list = []
mcc_list = []
rocauc_list = []

In [ ]:
model_list = [LogisticRegression(),DecisionTreeClassifier(),KNeighborsClassifier(n_neighbors=100),RandomForestClassifier(n_estimators=400),LGBMClassifier()]

In [ ]:
model = LogisticRegression(max_iter=300)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(model.score(X_test, y_test))
print(mt.f1_score(y_pred, y_test, zero_division=0,average='macro'))
print(mt.matthews_corrcoef(y_pred,y_test))
print(mt.roc_auc_score(y_train,model.predict_proba(X_train),multi_class='ovr'))

In [ ]:
for model in model_list:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy_list.append(model.score(X_test, y_test))
    f1_list.append(mt.f1_score(y_pred, y_test, zero_division=0,average='macro'))
    mcc_list.append(mt.matthews_corrcoef(y_pred,y_test))
    rocauc_list.append(mt.roc_auc_score(y_test,model.predict_proba(X_test),multi_class='ovr'))
    print('\nConfusion Matrix: ')
    print(mt.confusion_matrix(y_test,y_pred))
    print('\nConfusion Report: ') 
    print(mt.classification_report(y_test, y_pred))
    #rocauc_list.append(mt.roc_auc_score(y_test,model.predict_proba(X_test),multi_class='ovr'))

In [ ]:
print(accuracy_list,f1_list,mcc_list)

In [ ]:
# Showing results:
#y_pred=dtc.predict(X_test)
print('\nConfusion Matrix: ')
print(mt.confusion_matrix(y_test,y_pred))
print('\nConfusion Report: ') 
print(mt.classification_report(y_test, y_pred))